# Walkthrough of code

#### Authors:

Mustafa Hajij (did: NN architecture coding, evaluation metric coding)

Sierra Lear (did: downsampling function, running models using different hyperparameters, compiled/cleaned code into this Jupyter notebook)

Mahdi Moqri (did: all data collection and preprocessing)

## Loading and processing data



In [1]:
#import statements

import os
import numpy as np
import pandas as pd
import h5py
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from scipy.ndimage.filters import gaussian_filter
from PIL import Image
from scipy import ndimage

from sklearn.model_selection import train_test_split
from numpy import savetxt

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import InputLayer
from keras.utils import np_utils
from keras.datasets import mnist

import keras #Not needed?

from keras import backend as K
from keras.models import Model



import pickle

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [2]:
def myplot(x, y, s, bins=150):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)
    return heatmap.T

In [3]:
def load_point_cloud_from_hd(path):
    entiregraph=[]
    y=[]
    with open(path) as f:
         for line in f:
            numbers_str = line.split()
            numbers_float = [str(x) for x in numbers_str]  #map(float,numbers_str) works too
            val=float(numbers_float[-1])
            if val!=2:          
                y.append(val)
                entiregraph.append(numbers_float[:-1])
            else:
                y.append(1.0)
                entiregraph.append(numbers_float[:-1])
    return np.array(entiregraph),y

In [ ]:
def generate_data(expression_df,pairs,y_values,s=16,threshold=0.05,size=None):

    expression_df = expression_df.loc[:,~expression_df.columns.duplicated()]
    X=[]
    y_out=[]
    for pair,k in zip(pairs,y_values):
        
        if size!=None:
            
            if len(X)>size:
                break
        
        gene_a=pair[0]  
        gene_b=pair[1]
        
        if gene_a in expression_df and gene_b in expression_df : # check if the genes are in the table before plugging them in
            d=expression_df[[gene_a,gene_b]]
            d=d[d[gene_a]+d[gene_b]>threshold] # here we use the threshold 
        
        
            x=np.log2(d+1)[gene_a].values
            y=np.log2(d+1)[gene_b].values  


            
            if len(x)!=0:
                if len(x.shape)!=1:
                    
                    continue
                else:
                    
                      
                    img = myplot(x, y, s,bins=150) # create the image
                    
                    X.append(img)
                    y_out.append(k)
                    
                    
            else:
                
                    img = myplot(x, y, s,bins=150)  # create the image
             

                    X.append(img)
                    y_out.append(k)
              

        
    
    return np.array(X),np.array(y_out)    

The first step is to download all the raw data--namely single-cell expression data from this URL: https://mousescexpression.s3.amazonaws.com/dendritic_cell.h5.

In the cell below, I saved it as "dendritic_cell.h5."

In [ ]:
my_proj_dir = os.path.join(os.path.expanduser('~'), 'Documents/GitHub/deep_cell_personal/dendritic')

In [ ]:
os.path.join(my_proj_dir,'my_matrix.h5')

In [ ]:
folder= os.path.join(os.path.expanduser('~'), 'Documents/GitHub/deep_cell_personal/dendritic') #Note: must change to match location on your device
#genes_url='https://raw.githubusercontent.com/moqri/deep_cell/master/dendritic/genes.txt'
#labels_url='https://raw.githubusercontent.com/moqri/deep_cell/master/dendritic_gene_pairs_200.txt'

genes = pd.read_table(os.path.join(folder,'genes.txt'),index_col=1,header=None,names=['gene', 'id'],delimiter=' ') #references genes names and their id in raw data

expression_df = pd.read_hdf('~/Downloads/dendritic_cell.h5',index_col=0) #Note: this references/loads in the dendritic_cell raw data mentioned above. Again, location may have to be changed to match your device.

In [ ]:
#making sure genes downloaded correctly
genes.head()

In [ ]:
#making sure expression_df downloaded correctly
expression_df.head()

In [ ]:
#clean the data

expression_df.index.rename('cell_id',inplace=1)
expression_df.shape

gene_names=[genes['gene'].loc[id] for id in expression_df.columns.values] #adding appropriate gene names. Note: used loc instead ix because ix doesn't work on my machine.
expression_df.columns=gene_names 

expression_df.head()

expression_df=expression_df.loc[:, (expression_df != expression_df.iloc[0]).any()]  # remove constant columns
expression_df.shape

expression_df=expression_df[(expression_df.T != 0).any()] # remove rows of zeros
expression_df.shape


gene_count=expression_df.astype(bool).sum(axis=0)
cells=expression_df.count()
expression_df=expression_df[gene_count[gene_count>cells/10].index]
expression_df.shape

expression_df=(100*expression_df.transpose() / expression_df.sum(1)).round(2).transpose()
expression_df.head()


In [ ]:
#generate labels for different gene pairs from text file
labels= pd.read_table(folder+"/dendritic_gene_pairs_200.txt",index_col=False,header=None,names=['gene1', 'gene2','value'],delimiter='\t')
pairs,y_=load_point_cloud_from_hd(folder+"/dendritic_gene_pairs_200.txt")

labels.head()

In [ ]:
#Using expression dataframe and gene pairs+labels, create heatmaps of different gene pairs 
#and their associated label (0 = not correlated, 1 = correlated)
X,y=generate_data(expression_df,pairs,y_,s=16,threshold=0.00)

X_=X[:20000]
y_=y[:20000]

In [ ]:
#check that data looks good, i.e. X_ corresponds to matrix corresponding to pixel values of heatmap
                                 #y_ correspond to single-value label of 0 or 1
print(X_)
print(y_)

In [ ]:
#create new files containing the relevant training data--both the vectorized
with open('X_Data_all.pkl','wb') as f:
    pickle.dump(X_, f)

with open('y_Data_all.pkl','wb') as f:
    pickle.dump(y_, f)

## Developing model architecture and evaluation pipeline

Now that we have our training data, the next step is to:

1) develop our different neural net architectures

2) train our three different models using the training data developed in the section above


This section documents all the helper functions we developed in order to do this.


References for this section:
https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/

In [ ]:
#import statement

import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.models import Model

from sklearn.model_selection import train_test_split



# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from keras import metrics

import pickle

#from keras.layers import Input, Dense

import numpy as np

from keras.layers import InputLayer

from keras.layers import Conv2D

from keras.layers import Input, Dense

from matplotlib import pyplot

In [ ]:
#MODEL 1 ARCHITECTURE
def define_model_1(num_classes=2,shape_input=(150, 150, 1)):


    inputs = Input(shape=shape_input)

    x=Conv2D(32, kernel_size=(3, 3),
                     activation='relu')(inputs)
    x=Conv2D(64, (3, 3), activation='relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.25)(x)
    x=Flatten()(x)
    x=Dense(128, activation='relu')(x)
    x=Dropout(0.5)(x)
    predictions=Dense(num_classes, activation='softmax',name='final_output')(x)


    model = Model(inputs=inputs, outputs=predictions)
    print("model is defined")    
    return model

In [ ]:
#MODEL 2 ARCHITECTURE
def define_model_2(num_classes=2,shape_input=(150, 150, 1)):


    inputs = Input(shape=shape_input)

    x=Conv2D(16, kernel_size=(3, 3),activation='relu')(inputs)

    x=Conv2D(32, (3, 3), activation='relu')(x)
   
    x=Conv2D(64, (9, 9), activation='relu')(x)
  
    x=Conv2D(128, (17, 17), activation='relu')(x)    
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.25)(x)    

    x=Flatten()(x)
    x=Dense(128, activation='relu')(x)
    x=Dropout(0.5)(x)
    predictions=Dense(num_classes, activation='softmax',name='final_output')(x)


    model = Model(inputs=inputs, outputs=predictions)
    
    return model

In [ ]:
#MODEL 3 ARCHITECTURE
def define_model_3(num_classes,shape_input=(32, 32, 1)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=shape_input))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    print("model is defined")
    return model

In [ ]:
def train_model(model, x_train, y_train,x_test, y_test, batch_size=128,epochs=30,l_r=0.05,beta1=0.9,beta2=0.999):


    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=l_r,beta_1=beta1,beta_2=beta2),
                  metrics=[metrics.categorical_accuracy,metrics.mae])
    print("training the model")
    history=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

    print(history.history.keys())
    # summarize history for accuracy
    ##create plot showing accuracy over epochs on both training and test set
    plt.plot(history.history['categorical_accuracy']) #instead of 'acc'
    plt.plot(history.history['val_categorical_accuracy'])#instead of 'val_acc'
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    ##evaluation metrics of model at end
    print("done training the model")
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return history

In [ ]:
#splitting data prepped above into training and test sets
def prepare_data(X,y,test_size=0.05):        
    
    img_rows, img_cols = 150, 150
    
    # the data, split between train and test sets
    x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=test_size, random_state=42)


    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_test_sklearn=y_test
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, 2)
    y_test = keras.utils.to_categorical(y_test, 2)
    
    return x_train, y_train, x_test, y_test,y_test_sklearn  

In [ ]:
#function that returns even more possible evaluation metrics, just to be thorough
def get_scores_confusion_matrix_etc(model,x_test,y_test_sklearn):
    
    yhat_probs = model.predict(x_test, verbose=0)
    print(yhat_probs)
    
    
    def convert_2_sk_fommat(yhat_probs,y_test_sklearn):
        out=[]
        for i in range(0,len(yhat_probs)):
            if y_test_sklearn[i]==1:
                
                out.append(yhat_probs[0][1])
            else:
                out.append(yhat_probs[0][0])
        return out   
                
    
    # predict crisp classes for test set
    yhat_classes = yhat_probs.argmax(axis=1)    
    print(yhat_classes)
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test_sklearn, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test_sklearn, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test_sklearn, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test_sklearn, yhat_classes)
    print('F1 score: %f' % f1)    

    # kappa
    kappa = cohen_kappa_score(y_test_sklearn, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    yhat_probs_sklearn=convert_2_sk_fommat(yhat_probs,y_test_sklearn)
    
    auc = roc_auc_score(y_test_sklearn, yhat_probs_sklearn)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("the confusion matrix : ")
    matrix = confusion_matrix(y_test_sklearn, yhat_classes)
    print(matrix)
    
    return yhat_probs_sklearn, y_test_sklearn,matrix,f1,recall,precision,accuracy

In [ ]:
 def plot_loss_accuracy(history):
    pyplot.subplot(211)
    pyplot.title('Loss')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    # plot accuracy during training
    pyplot.subplot(212)
    pyplot.title('Accuracy')
    pyplot.plot(history.history['categorical_accuracy'], label='train')
    pyplot.plot(history.history['val_categorical_accuracy'], label='test')
    pyplot.legend()
    pyplot.show()   



While training our model, we noted some that our heavily unbalanced dataset was causing our neural nets to be trained to always output the majority class. To combat this, we created a downsampling helper function to correct for data imbalance.

Reference for helper function:
https://chrisalbon.com/machine_learning/preprocessing_structured_data/handling_imbalanced_classes_with_downsampling/

In [ ]:
new_folder = '/Users/sierra.lear/Documents/GitHub/deep_cell_personal'

with open(new_folder+'/X_Data_all.pkl','rb') as f:
    X_ = pickle.load(f)
    print(X_.shape)

with open(new_folder+'/y_Data_all.pkl','rb') as f:
    y_ = pickle.load(f)
    print(y_.shape)

In [ ]:
y_equals_0 = np.where(y_ == 0)[0]
y_equals_1 = np.where(y_ == 1)[0]

print("Number of total data points:", len(y_))
print("Number of data points with label 0:",len(y_equals_0))
print("Number of data points with label 1:",len(y_equals_1))

In [ ]:
def downsampling_data(X_, y_):
    np.random.seed(8)
    #Find the indices for the 0 and 1 class
    i_0 = np.where(y_ == 0)[0]
    i_1 = np.where(y_ == 1)[0]
    #count number of observations in each class
    n_0 = len(i_0)
    n_1 = len(i_1)
    #for every observation in class 0 (small class size), randomly sample from class 1
    i_1_downsample = np.random.choice(i_1, size = n_0, replace=False)
    #recreate X_ and y_ with only downsampled values
    X_downsampled = np.concatenate((X_[i_0,:,:], X_[i_1_downsample,:,:]), axis=0)
    y_downsampled = np.hstack((y_[i_0], y_[i_1_downsample]))
    
    return X_downsampled, y_downsampled

In [ ]:
X_down, y_down = downsampling_data(X_, y_)
print("Number of total data points:", len(y_down))
print("Number of data points with label 0:",len(np.where(y_down == 0)[0]))
print("Number of data points with label 1:",len(np.where(y_down == 1)[0]))

## Training the models

In [ ]:
#load the data
new_folder = '/Users/sierra.lear/Documents/GitHub/deep_cell_personal'

with open(new_folder+'/X_Data_all.pkl','rb') as f:
    X_ = pickle.load(f)
    print(X_.shape)

with open(new_folder+'/y_Data_all.pkl','rb') as f:
    y_ = pickle.load(f)
    print(y_.shape)

In [ ]:
#downsample the data to create a balanced dataset
X_down, y_down = downsampling_data(X_, y_)

In [ ]:
#prepare the training and test sets from the balanced downsampled dataset
x_train, y_train, x_test, y_test,y_test_sklearn = prepare_data(X_down,y_down)

In [ ]:
"""
the standard pipeline we will be using to train the models
"""

##(1) define the model: have define_model_1, define_model_2, and define_model 3 to choose from 
#model=define_model_3(num_classes=2,shape_input=(150, 150, 1))

##(2) fit the model   -- we can also play with different hyperparameters here
#history=train_model(model, x_train, y_train,x_test, y_test, batch_size=32,epochs=2,l_r=0.005,beta1=0.9,beta2=0.999)

##(3) plot history :
#plot_loss_accuracy(history)

##(4) get accuracy,  f1 score, confusion matrix, etc.   
#yhat_probs_sklearn, y_test_sklearn,matrix,f1,recall,precision,accuracy=get_scores_confusion_matrix_etc(model,x_test,y_test_sklearn)




"""

to plot ROC :
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py    

"""







For the first pass, I will use all the data, 2 epochs, a batch size of 32 to make sure everything works. Afterwards, I will be changing both learning rate and epoch size as my two hyperparameters of choice.

In [ ]:
#(1) define the model: have define_model_1, define_model_2, and define_model 3 to choose from 
model=define_model_1(num_classes=2,shape_input=(150, 150, 1))

#(2) fit the model   -- we can also play with different hyperparameters here
history=train_model(model, x_train, y_train,x_test, y_test, batch_size=32,epochs=2,l_r=0.005,beta1=0.9,beta2=0.999)

#(3) plot history :
plot_loss_accuracy(history)

#(4) get accuracy,  f1 score, confusion matrix, etc.   
yhat_probs_sklearn, y_test_sklearn,matrix,f1,recall,precision,accuracy=get_scores_confusion_matrix_etc(model,x_test,y_test_sklearn)


In [ ]:
# pipeline 

#(1) load the data    



with open('X_Data_all.pkl','rb') as f:
    X_ = pickle.load(f)
    print(X_.shape)

with open('y_Data_all.pkl','rb') as f:
    y_ = pickle.load(f)
    print(y_.shape)
    
X = X_[:100]
Y = y_[:100]



x_train, y_train, x_test, y_test,y_test_sklearn = prepare_data(X,Y)


#(2) define the model   


model=define_model_1(num_classes=2,shape_input=(150, 150, 1))



# #(3) fit the model   

# history=train_model(model, x_train, y_train,x_test, y_test, batch_size=32,epochs=2,l_r=0.005,beta1=0.9,beta2=0.999)


# # (4) plot history :

# plot_loss_accuracy(history)



# #(5) get accurcy,  f1 score, confusion matrix, etc   



# yhat_probs_sklearn, y_test_sklearn,matrix,f1,recall,precision,accuracy=get_scores_confusion_matrix_etc(model,x_test,y_test_sklearn)



In [ ]:

model=define_model_2(num_classes=2,shape_input=(150, 150, 1))

In [ ]:


#(3) fit the model   

history=train_model(model, x_train, y_train,x_test, y_test, batch_size=32,epochs=2,l_r=0.005,beta1=0.9,beta2=0.999)


# (4) plot history :

plot_loss_accuracy(history)



#(5) get accurcy,  f1 score, confusion matrix, etc   



yhat_probs_sklearn, y_test_sklearn,matrix,f1,recall,precision,accuracy=get_scores_confusion_matrix_etc(model,x_test,y_test_sklearn)



In [ ]:
yhat_probs_sklearn, y_test_sklearn,matrix,f1,recall,precision,accuracy=get_scores_confusion_matrix_etc(model,x_test,y_test_sklearn)